In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype
import pickle
import ast

**FINAL RESULTS :
RMSE on train2 with training on train1 = 2.308427215577763
RMSE on test with training on train1+train2 = 2.401714997765819**

**Filter out places having a specific Lat-Long range**

In [ ]:
index_list = []
for index,row in merged_df.iterrows():
    if float(row["Latitude"]) > 10**7 or float(row["Latitude"]) < -10**7:
        index_list.append(index)

**Review Preprocessing**

In [2]:
df_reviews = pd.read_csv("/Users/studentuser/Downloads/Project/Reviews.csv")

In [3]:
places_categories = pd.read_csv("places_with_categories.csv")
places_categories.columns = ['gPlusPlaceId', 'name', 'Latitude', 'Longitude', 'categories']

In [4]:
merged_df = df_reviews.merge(places_categories, on = 'gPlusPlaceId')
merged_df = merged_df.dropna(subset=["categories_y"])

**Since gPlusPlaceId and gPlusUserId are long strings, there was a mapping created to smaller indices which was stored in pickles (new_item.pickle and new_user.pickle)**

In [84]:
item_pickle = open("/Users/studentuser/Downloads/Project/pickles/new_item.pickle","rb")
item_dict = pickle.load(item_pickle)

In [85]:
user_pickle = open("/Users/studentuser/Downloads/Project/pickles/new_user.pickle","rb")
user_dict = pickle.load(user_pickle)

In [8]:
merged_df["placeIndex"] = merged_df["gPlusPlaceId"].apply(lambda x : item_dict[x] if x in item_dict.keys() else None )
merged_df["userIndex"] = merged_df["gPlusUserId"].apply(lambda x : user_dict[x] if x in user_dict.keys() else None)

In [9]:
merged_df = merged_df.dropna(subset=['placeIndex', 'userIndex'])

**Create two dictionaries** 
1. One dictionary will be dict[place] = ["Asian" , "Chinese"]
2. Second dictionary will be dict["Asian"] = ["place1" , "place2"]

In [10]:
place_to_cat_pickle = open("/Users/studentuser/Downloads/Project/pickles/place_to_cat_new.pickle","rb")
place_to_cat = pickle.load(place_to_cat_pickle)

In [11]:
cat_to_place_pickle = open("/Users/studentuser/Downloads/Project/pickles/cat_to_place_new.pickle","rb")
cat_to_place = pickle.load(cat_to_place_pickle)

In [12]:
merged_df.shape

(18798, 13)

In [13]:
merged_df.head()

,Unnamed: 0,categories_x,gPlusPlaceId,gPlusUserId,rating,reviewTime,unixReviewTime,name,Latitude,Longitude,categories_y,placeIndex,userIndex
95,22,[u'Market'],103133677716773897224,100000039815955317902,4.0,"Dec 14, 2012",1.355475e+09,Moirabari Market,26.454098,92.421952,[u'Market'],974.0,0.0
96,1329667,[u'Market'],103133677716773897224,102118273302242190955,3.0,"Aug 30, 2012",1.346393e+09,Moirabari Market,26.454098,92.421952,[u'Market'],974.0,1293.0
97,2330330,[u'Market'],103133677716773897224,103707354936917456863,2.0,"Mar 18, 2013",1.363634e+09,Moirabari Market,26.454098,92.421952,[u'Market'],974.0,2323.0
98,8706625,[u'Market'],103133677716773897224,113956358126441896410,3.0,"Nov 19, 2012",1.353317e+09,Moirabari Market,26.454098,92.421952,[u'Market'],974.0,8949.0
99,11171878,[u'Market'],103133677716773897224,117980982174626534554,5.0,"Jan 19, 2014",1.390137e+09,Moirabari Market,26.454098,92.421952,[u'Market'],974.0,11452.0


**Create a dataframe to store place info with their geolocation and categories**

In [27]:
place_geodf = merged_df_final[['placeIndex','Latitude', 'Longitude']].copy()
place_geodf = place_geodf.set_index('placeIndex')
place_geodf = place_geodf.loc[~place_geodf.index.duplicated(keep='first')]
place_geodf.head()

,Latitude,Longitude
placeIndex,,
974.0,26.454098,92.421952
459.0,26.782564,83.409703
2204.0,26.484356,90.548533
2943.0,27.973463,85.963812
2057.0,26.792200,88.448716


In [342]:
place_geodf.to_csv("place_geo_final.csv")

Sanity check 

In [28]:
place_geodf.loc[459,]

Latitude     26.782564
Longitude    83.409703
Name: 459.0, dtype: float64

In [341]:
merged_df[merged_df["placeIndex"]== 215]

,Unnamed: 0,categories_x,gPlusPlaceId,gPlusUserId,rating,reviewTime,unixReviewTime,name,Latitude,Longitude,categories_y,placeIndex,userIndex
8707432,2970481,[u'Hindu Temple'],100699313239966093888,104736345918291046305,4.0,"Feb 10, 2013",1.360547e+09,Gokarna Mahadev Temple,27.739403,85.387807,[u'Hindu Temple'],215.0,2981.0
8707433,4078970,[u'Hindu Temple'],100699313239966093888,106516869171611562978,5.0,"Nov 2, 2013",1.383409e+09,Gokarna Mahadev Temple,27.739403,85.387807,[u'Hindu Temple'],215.0,4127.0
8707434,4399352,[u'Hindu Temple'],100699313239966093888,107033935968639228641,5.0,"Mar 24, 2013",1.364170e+09,Gokarna Mahadev Temple,27.739403,85.387807,[u'Hindu Temple'],215.0,4477.0


**Do place pre-processing**

**Get number of distinct categories in the full dataset**

In [345]:
len(cat_to_place.keys())

897

**Generate rating for all users in train2**  (user : [[item,rating,time],[item,rating,time],[item,rating,time]])

In [376]:
train2_pickle = open("/Users/studentuser/Downloads/Project/pickles/train2_asia.pickle","rb")
train2 = pickle.load(train2_pickle)

**Write a function to return the placeIndex for the given row**

In [347]:
def my_func(x):
    return merged_df_final.iloc[x,].placeIndex


**Find the rating predictions for each user, this would be done as follows :**

1. Compute the mean rating given by the user
2. Find the places where the overall rating is above the mean rating given by the user
3. Find the categories for all such places
4. For each category from the categories above, find the place and if not visited by the user , add it t the contender places list


**Sample code to check the working of geopy.distance**

In [69]:
import geopy.distance

coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print(geopy.distance.vincenty(coords_1, coords_2).km)

279.35290160386563


/Users/studentuser/anaconda2/envs/mypython3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  


**Read the user profile csv and get the location related information for each user**

In [106]:
user_profile = pd.read_csv("/Users/studentuser/Downloads/Project/user_profiles_new.csv")
len(user_profile)

8777

In [30]:
from scipy import sparse
train1_asia = sparse.load_npz("/Users/studentuser/Downloads/Project/sparse_matrix_train1_asia.npz")

In [136]:
from scipy import sparse
train1_time_asia = sparse.load_npz("/Users/studentuser/Downloads/Project/sparse_matrix_train1_time_asia.npz")

In [65]:
train2_asia = open("/Users/studentuser/Downloads/Project/pickles/train2_asia.pickle","rb")
train2_asia = pickle.load(train2_asia)

In [82]:
test_asia = open("/Users/studentuser/Downloads/Project/pickles/test_asia.pickle","rb")
test_asia = pickle.load(test_asia)

In [321]:
from scipy import sparse
train12_asia = sparse.load_npz("/Users/studentuser/Downloads/Project/sparse_matrix_train1_asia_trian1+train2.npz")

In [323]:
from scipy import sparse
train12_time_asia = sparse.load_npz("/Users/studentuser/Downloads/Project/sparse_matrix_train1_time_asia_trian1+train2.npz")

In [324]:
# Create a dictionary to store mappings of user to list of (contender_place,distance)
user_contender_dist = {}
predict_global_mean = []
predict_zero_rating = []
predict_five_rating = []
counter = 0
for user,val in test_asia.items():
    counter += 1
    if counter % 100 == 0:
        print("Done processing " , counter , " users")
    user_loc = tuple()
    
    if user in user_profile.index:
        if not np.isnan(user_profile.loc[user]["current_lat"]):
            user_loc = (user_profile.loc[user]["current_lat"],user_profile.loc[user]["current_long"])
        elif not np.isnan(user_profile.loc[user]["previous_lat"]):
            user_loc = (user_profile.loc[user]["previous_lat"],user_profile.loc[user]["previous_long"])

    # First check if user location information is present, if not present then find the location of the most recently 
    # reviewed place and make that the user location
    if len(user_loc) == 0:
        if len(train12_time_asia.getrow(user).tocoo().data) > 0:
            max_timestamp = 0
            for j,v in zip(train12_time_asia.getrow(user).tocoo().col,train12_time_asia.getrow(user).tocoo().data):
                if v > max_timestamp:
                    latest_place = j
            if latest_place in place_geodf.index:
                user_loc = (place_geodf.loc[latest_place]["Latitude"],place_geodf.loc[latest_place]["Longitude"])
        else:
            # Such a user doesn't have any location or rating information and we would predict global mean for all
            # places he visits in train2
            predict_global_mean.append(user)



    if len(user_loc) > 0:
        # Check if user has some rating in train1_asia, if there are ratings then compute the mean of the ratings
        # and then find the categories for which the user has rated the places above the mean rating
        # if there are no ratings then the contender places would be the nearby places

        contender_places = set()
        if len(train12_asia.getrow(user).tocoo().data) > 0:

            mean_rating_user = np.mean(train12_asia.getrow(user).tocoo().data)
            places_above_mean = []
            for j,v in zip(train12_asia.getrow(user).tocoo().col,train12_asia.getrow(user).tocoo().data):

                if v > mean_rating_user - 0.2:
                    places_above_mean.append(j)


            for place in places_above_mean:
                for cat in place_to_cat[place]:
                    for contender_place in cat_to_place[cat]:
                        if contender_place not in places_above_mean:
                            contender_places.add(contender_place)
                            
                            
            if len(contender_places) == 0:
                predict_zero_rating.append(user)
                            
            elif len(contender_places) == 1:
                predict_five_rating.append(user)
                
            else:
                for place in contender_places:
                    if place in place_geodf.index:
                        if not np.isnan(place_geodf.loc[place]["Latitude"]):

                            place_loc = (place_geodf.loc[place]["Latitude"] , place_geodf.loc[place]["Longitude"])
                            dist = geopy.distance.geodesic(user_loc, place_loc ).km
                            if user in user_contender_dist:
                                user_contender_dist[user].append((place,dist))
                            else:
                                user_contender_dist[user] = []
                                user_contender_dist[user].append((place,dist))
            


        else:
            maxLat = user_loc[0] + 4
            minLat = user_loc[0] - 4
            maxLong = user_loc[1] + 4
            minLong = user_loc[1] - 4
            # add and subtract 2 to the lat long
            fabricated_places = 0
            for place in place_geodf.index:
                if (place_geodf.loc[place]["Latitude"] < maxLat and place_geodf.loc[place]["Latitude"] > minLat) and \
                    (place_geodf.loc[place]["Longitude"] < maxLong and place_geodf.loc[place]["Longitude"] > minLong ):

                    place_loc = (place_geodf.loc[place]["Latitude"] , place_geodf.loc[place]["Longitude"])
                    dist = geopy.distance.geodesic(user_loc, place_loc ).km
                    fabricated_places += 1
                    if user in user_contender_dist:
                        user_contender_dist[user].append((place,dist))
                    else:
                        user_contender_dist[user] = []
                        user_contender_dist[user].append((place,dist))
                        
            if fabricated_places == 0:
                predict_zero_rating.append(user)

    #Normalize distances for each user and concert distances to rating

Done processing  100  users
Done processing  200  users
Done processing  300  users
Done processing  400  users
Done processing  500  users
Done processing  600  users
Done processing  700  users
Done processing  800  users
Done processing  900  users
Done processing  1000  users
Done processing  1100  users
Done processing  1200  users
Done processing  1300  users
Done processing  1400  users
Done processing  1500  users
Done processing  1600  users
Done processing  1700  users
Done processing  1800  users
Done processing  1900  users
Done processing  2000  users
Done processing  2100  users
Done processing  2200  users
Done processing  2300  users
Done processing  2400  users
Done processing  2500  users
Done processing  2600  users


**Compute Normalized Ratings for contender places using the computed distances**

**Split the dictionary into cold start users and actual users**

Print those users who have given a rating, but there are no contender places corresponding to the categories in which they gave the ratings.

In [300]:
for user in train2_asia.keys():
    if user not in predict_global_mean and user not in predict_five_rating and user not in predict_zero_rating:
        if user not in user_contender_dist.keys():
            print(user)
    

In [188]:
train1_asia.getrow(58).tocoo().data

array([], dtype=float64)

Sanity check #1  : Check if the users in user_contender_dist have atleast two contender places

In [272]:
for user in user_contender_dist:
    if len(user_contender_dist[user]) < 2:
        print("Hello")

**Compute Normalized Ratings for contender places using the computed distances. This would be done for users who have greater than or equal to two contender places**

In [345]:
user_contender_preds = {}

for user in user_contender_dist:
    
    
    if user not in predict_global_mean and user not in predict_five_rating and user not in predict_zero_rating:
        
        contender_places = user_contender_dist[user]

        if(len(contender_places) > 2):
            max_dist = 0
            for place in contender_places:
                if place[1] > max_dist:
                    max_dist = place[1]

            min_dist = float('inf')
            for place in contender_places:
                if place[1] < min_dist:
                    min_dist = place[1]

            for place in contender_places:
                score = ((place[1]-min_dist)/(max_dist-min_dist))
                pred = (1-score)*5.0
                user_contender_preds[(user,place[0])] = pred

**Calculate Global Mean Rating for a given sparse matrix**

In [332]:
tot_item_rating = 0
tot_item_count = 0
for user, item, rating_here in zip(train12_asia.row, train12_asia.col, train12_asia.data):

    tot_item_rating += rating_here
    tot_item_count += 1
    counter+=1


    
global_mean = tot_item_rating/tot_item_count

In [333]:
global_mean

4.000200226923847

In [347]:
#Now go through the train2 data and make predictions and calculate RMSE
final_preds = {}
sum_rmse = 0
N = 0
for user,val in test_asia.items():
    
    if user in predict_global_mean:
        for item in val:
            N += 1
            final_preds[(user,item[0])] = global_mean
            sum_rmse += (global_mean - item[1])**2
    elif user in predict_five_rating:
        for item in val:
            N += 1
            final_preds[(user,item[0])] = 5.0
            sum_rmse += (5 - item[1])**2
    elif user in predict_zero_rating:
        for item in val:
            N += 1
            final_preds[(user,item[0])] = 0  
            sum_rmse += (item[1])**2
    else:
        for item in val:
            N += 1
            if (user,item[0]) in user_contender_preds:
                prediction = user_contender_preds[(user,item[0])]
                final_preds[(user,item[0])] = prediction
                sum_rmse += (prediction - item[1])**2
            else:
                final_preds[(user,item[0])] = 0
                sum_rmse += (item[1])**2
                
            
        
        
    

In [352]:
np.sqrt(sum_rmse/N)

2.401714997765819

In [356]:
with open('content_based_preds_test_combined.pickle', 'wb') as handle:
    pickle.dump(final_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [357]:
content_based_preds_test_cold_start = {}
content_based_preds_test_only = {}
for key,val in final_preds.items():
    if key[0] in predict_global_mean or key[0] in predict_zero_rating:
        content_based_preds_test_cold_start[key] = val
    else:
        content_based_preds_test_only[key] = val
        
    

In [358]:
with open('content_based_preds_test_cold_start.pickle', 'wb') as handle:
    pickle.dump(content_based_preds_test_cold_start, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [359]:
with open('content_based_preds_test_only.pickle', 'wb') as handle:
    pickle.dump(content_based_preds_test_only, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [311]:
np.sqrt(sum_rmse/N)

2.308427215577763

**Remove corrupt values of Latitude and Longitude**

In [15]:
merged_df = merged_df.drop(merged_df[(merged_df.Latitude < 10**7) & (merged_df.Latitude > 90)].index)
merged_df = merged_df.drop(merged_df[(merged_df.Latitude > -10**7) & (merged_df.Latitude < -90)].index)

In [16]:
len(merged_df)

18798

**Create a subset of lat-long that are corrupt but can be recovered**

In [17]:
merged_df_7 = merged_df.loc[(merged_df['Latitude'] >= 10**7) | (merged_df['Latitude'] <= -10**7)]
merged_df_not_7 = merged_df.loc[~((merged_df['Latitude'] >= 10**7) | (merged_df['Latitude'] <= -10**7))]

In [18]:
#index_list = []
lat_list = []
long_list = []
counter = 0
for index,row in merged_df_7.iterrows():
    if counter % 100 == 0:
        print("Done with " , counter,  " rows")
    lat_list.append(float(row["Latitude"])/(10**7))
    long_list.append(float(row["Longitude"])/(10**7))
    counter += 1

In [19]:
merged_df_7["Latitude"] = lat_list
merged_df_7["Longitude"] = long_list

In [22]:
merged_df_final = pd.concat([merged_df_7, merged_df_not_7])

**Using first and second mapping, create the final mapping for gplusUserId:userIndex**

In [94]:
item_gplus_old_pickle = open("/Users/studentuser/Downloads/Project/item_mapping_dict_from_category.pickle","rb")
item_gplus_old_dict= pickle.load(item_gplus_old_pickle)


In [95]:
user_gplus_old_pickle = open("/Users/studentuser/Downloads/Project/pickles/user_mapping_dict_from_category.pickle","rb")
user_gplus_old_dict = pickle.load(user_gplus_old_pickle)


In [96]:
item_old_new_pickle = open("/Users/studentuser/Downloads/Project/pickles/item_mapping_dict_from_category_old_new.pickle","rb")
item_old_new_dict = pickle.load(item_old_new_pickle)

In [97]:
user_old_new_pickle = open("/Users/studentuser/Downloads/Project/pickles/user_mapping_dict_old_new.pickle","rb")
user_old_new_dict = pickle.load(user_old_new_pickle)

In [98]:
user_dict = {}
item_dict = {}


for item,val in item_gplus_old_dict.items():
    if val in item_old_new_dict.keys():
        item_dict[item] = item_old_new_dict[val]
        
for user,val in user_gplus_old_dict.items():
    if val in user_old_new_dict.keys():
        user_dict[user] = user_old_new_dict[val]
        
